In [ ]:
#install required packages
import numpy as np
import pandas as pd
from plotnine import *
from datetime import datetime

## Question 1: End to End Time

In [ ]:
def process_and_clean_files():
    for month in range(1, 13):
        input_file = f"___.csv"
        output_file = f"____.csv"

        print(f'Processing file: {input_file}')

        # Read CSV file into a DataFrame
        df = pd.read_csv(input_file)

        # Keep only the required columns
        df = df[['route_id', 'stop_id', 'direction_id', 'half_trip_id', 'point_type', 'scheduled', 'actual']]

        # Keep only rows where 'point_type' is 'Startpoint' or 'Endpoint'
        #df = df[df['point_type'].isin(['Startpoint', 'Endpoint'])]

        # Convert time format columns
        df['scheduled'] = pd.to_datetime(df['scheduled']).dt.strftime('%H:%M:%S')
        df['actual'] = pd.to_datetime(df['actual']).dt.strftime('%H:%M:%S')

        # Order rows
        df = df.sort_values(by=['route_id', 'half_trip_id'], ascending=[True, True])

        # Save the cleaned DataFrame to a new CSV file
        df.to_csv(output_file, index=False)
        print(f'Cleaned file saved as: {output_file}')

process_and_clean_files()

In [ ]:
def process_and_calculate_end_to_end_times():
    for month in range(1, 13):
        input_file = f"___.csv"
        output_file = f"___.csv"

        print(f"Processing file: {input_file}")

        # Read the file into a DataFrame
        df = pd.read_csv(input_file)

        # Filter out rows where point_type is 'Startpoint' or 'Endpoint'
        startpoint = df[df['point_type'] == 'Startpoint']
        endpoint = df[df['point_type'] == 'Endpoint']

        # Merge startpoint and endpoint data on 'route_id', 'direction_id', and 'half_trip_id'
        merged = pd.merge(startpoint, endpoint, on=['route_id', 'direction_id', 'half_trip_id'], suffixes=('_start', '_end'))

        # Convert time columns to datetime format
        merged['scheduled_start'] = pd.to_datetime(merged['scheduled_start'])
        merged['scheduled_end'] = pd.to_datetime(merged['scheduled_end'])
        merged['actual_start'] = pd.to_datetime(merged['actual_start'])
        merged['actual_end'] = pd.to_datetime(merged['actual_end'])

        # Calculate end-to-end time by subtracting scheduled times
        merged['scheduled_end_to_end'] = (merged['scheduled_end'] - merged['scheduled_start']).dt.total_seconds() / 60
        merged['actual_end_to_end'] = (merged['actual_end'] - merged['actual_start']).dt.total_seconds() / 60

        # Adjust end-to-end time for cases where end time is before start time (crossing over midnight)
        merged['scheduled_end_to_end'] = merged.apply(lambda row: row['scheduled_end_to_end'] + 1440 if row['scheduled_end'] < row['scheduled_start'] else row['scheduled_end_to_end'], axis=1)
        merged['actual_end_to_end'] = merged.apply(lambda row: row['actual_end_to_end'] + 1440 if row['actual_end'] < row['actual_start'] else row['actual_end_to_end'], axis=1)

        # Drop rows with missing/null values in 'actual_end_to_end' column
        merged.dropna(subset=['actual_end_to_end'], inplace=True)

        # Calculate the difference between scheduled and actual end-to-end times
        merged['difference'] = merged['scheduled_end_to_end'] - merged['actual_end_to_end']

        # Select necessary columns
        result = merged[['route_id', 'direction_id', 'half_trip_id', 'scheduled_end_to_end', 'actual_end_to_end', 'difference']]

        # Save the result to a new file
        result.to_csv(output_file, index=False)
        print(f"Result saved to {output_file}\n")

# Process and calculate end-to-end times for all files
process_and_calculate_end_to_end_times()

## Question 2: Route Lateness

In [ ]:
def process_and_calculate_route_average_lateness():
    for month in range(1, 13):
        input_file = f"___.csv"
        output_file = f"___.csv"

        print(f"Processing file: {input_file}")

        # Read the dataset
        df = pd.read_csv(input_file)

        # Convert time columns to datetime format
        df['scheduled'] = pd.to_datetime(df['scheduled'])
        df['actual'] = pd.to_datetime(df['actual'])

        # Calculate lateness
        df['lateness'] = abs((df['scheduled'] - df['actual']).dt.total_seconds() / 60)  # Convert to minutes

        # Group by 'route_id' and calculate average lateness across all stops
        route_average_lateness = df.groupby('route_id')['lateness'].mean().reset_index()
        route_average_lateness.rename(columns={'lateness': 'average_lateness'}, inplace=True)

        # Save the result to a new file
        route_average_lateness.to_csv(output_file, index=False)
        print(f"Average lateness per route for month {month} saved to {output_file}\n")

# Process and calculate route average lateness for all files
process_and_calculate_route_average_lateness()
